In [1]:
pip install tiktoken -q

In [ ]:
# converting text to tokens

import tiktoken
import numpy as np

tokenizer=tiktoken.get_encoding('cl100k_base')

text="Hello world"
tokens=tokenizer.encode(text)
print(f"tokens -> {tokens}")

for token in tokens:
    print(f"{token} -> '{tokenizer.decode([token])}'")

tokens -> [9906, 1917]
9906 -> 'Hello'
1917 -> ' world'


In [3]:
%pip install gensim -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 50.1 MB/s eta 0:00:00:00:0100:01


In [4]:
# getting vectors for tokens

import gensim.downloader as api

word_vectors=api.load('glove-wiki-gigaword-100')

print(f"loaded vocablury of size {len(word_vectors)} with {len(word_vectors[1])} dimensions")

[==================================================] 100.0% 128.1/128.1MB downloaded
loaded vocablury of size 400000 with 100 dimensions


In [61]:
# positional encoding function
dimensions=len(word_vectors[1])

def positinal_encoding(pos,dim):
    pos_embed=[]
    for i in range(dim):
        if(i%2==0):
            pos_embed.append(np.sin(pos/10000**(i/dim)))
        else:
            pos_embed.append(np.cos(pos/10000**((i-1)/dim)))
    return pos_embed

In [62]:
# adding embeddings and positional embeddings


embeddings=[]

for token in tokens:
    embeddings.append(word_vectors[token])


final_embeddings=np.array(embeddings)

for i in range(len(tokens)):
    print(positinal_encoding(i,dimensions))
    final_embeddings+=positinal_encoding(i,dimensions)




[np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.

In [ ]:
# softmax function

def softmax(mat):
    for i in range(len(mat)):
        total=0
        for j in range(len(mat[i])):
            total+=np.exp(mat[i][j])
        for j in range(len(mat[i])):
            mat[i][j]=np.exp(mat[i][j])/total

In [ ]:
# function for calculating attention (q,k,v)

def attention(embeds,w_q,w_k,w_v):
    q=embeds@w_q
    k=embeds@w_k
    v=embeds@w_v
    similarity=q@(k.T)
    norm_sim=similarity/(np.sqrt(len(w_k)))
    softmax(norm_sim)
    atten=norm_sim@v
    return atten

In [ ]:
# calculating attention

w_q=np.random.randn(dimensions,50)*0.02
w_k=np.random.randn(dimensions,50)*0.02
w_v=np.random.randn(dimensions,50)*0.02

final=attention(final_embeddings,w_q,w_k,w_v)

print((final))

[[-0.19336625 -0.08678693]
 [-0.31437599 -0.0151426 ]]
[[-0.01933662 -0.00867869]
 [-0.0314376  -0.00151426]]
[[ 1.10796934e-01 -1.91302597e-01 -1.41070092e-01  4.83278334e-02
   7.43794710e-02 -3.52963880e-01  2.73491204e-01  3.91234897e-01
  -2.46095653e-01 -3.66569764e-01  8.24507841e-02  6.40498675e-02
  -4.14513870e-01 -2.39685206e-01  2.42724642e-02  7.73355128e-01
   2.45479834e-01  1.19980016e-01  1.37784364e-01  2.83959962e-02
   5.06989634e-01 -1.12831033e-03  1.14671407e-01  2.75081804e-02
  -3.93976864e-01  3.54964332e-01  2.81997899e-01 -3.13398700e-02
   9.63908022e-04 -1.32057829e-01 -1.28087077e-01  6.30807108e-01
   5.69809654e-02 -8.20531891e-01  4.74280614e-01 -9.41359603e-02
   3.02161473e-01 -9.36021577e-02 -3.00721357e-01  2.71492169e-01
  -1.23384111e-01  5.26108997e-03 -5.44143754e-01  5.21745175e-01
   7.43250546e-02  6.46707129e-01 -3.36718090e-02 -9.47982141e-02
   1.10952227e-01 -1.18617485e-01]
 [ 1.12326164e-01 -1.91908186e-01 -1.40574814e-01  4.72087418e-